# Praktische dingen voor een admin

## Logfile analyse

De file *test.log* bevat de volgende informatie:
```
Sep 16 10:38:45 yoram-thinkpad kernel: [ 1948.146436] CPU0: Package temperature/speed normal
Sep 16 10:38:49 yoram-thinkpad kernel: [ 1952.421197] mce: [Hardware Error]: Machine check events logged
Sep 16 10:39:57 yoram-thinkpad wpa_supplicant[3370]: wlan0: CTRL-EVENT-SCAN-STARTED 
-------- etc ---------
```

In [1]:
filehandle = open('test.log', 'r')
filelines = filehandle.readlines()
filehandle.close()
for line in filelines:
    print line, # de laatste ',' betekent dat er geen newline achter elke line geplaatst wordt

Sep 16 10:38:45 yoram-thinkpad kernel: [ 1948.146436] CPU0: Package temperature/speed normal
Sep 16 10:38:49 yoram-thinkpad kernel: [ 1952.421197] mce: [Hardware Error]: Machine check events logged
Sep 16 10:39:57 yoram-thinkpad wpa_supplicant[3370]: wlan0: CTRL-EVENT-SCAN-STARTED 


Ervanuit gaande dat de log een vaste structuur heeft, kunnen we deze makkelijk splitsen/parsen

In [2]:
for line in filelines:
    # Eerst maken we van de regel een lijst
    line_list = line.split()
    
    # Item 0 t/m 3 is de datum.
    date_string = ' '.join(line_list[:3])
    
    hostname = line_list[3]
    
    # Vanaf het 5e item begint de message
    message = ' '.join(line_list[4:])
    print "date: {}\nhostname: {}\nmessage: {}\n---------------".format(date_string, hostname, message)

date: Sep 16 10:38:45
hostname: yoram-thinkpad
message: kernel: [ 1948.146436] CPU0: Package temperature/speed normal
---------------
date: Sep 16 10:38:49
hostname: yoram-thinkpad
message: kernel: [ 1952.421197] mce: [Hardware Error]: Machine check events logged
---------------
date: Sep 16 10:39:57
hostname: yoram-thinkpad
message: wpa_supplicant[3370]: wlan0: CTRL-EVENT-SCAN-STARTED
---------------


In [3]:
from datetime import datetime
date_string = 'Sep 16 10:38:45'
date_object = datetime.strptime(date_string, '%b %d %H:%M:%S')
# We hebben alleen het jaar niet gespecificeerd. Dit staat standaar op 1900
date_object = date_object.replace(year=2015)
print date_object

2015-09-16 10:38:45


In [4]:
# We kunnen nu ook makkelijk delta's bekijken
verschil = datetime.now() - date_object
print "aantal seconden: {}".format(verschil.seconds)

# Of, als we fancy willen doen
minuten, seconden = divmod(verschil.seconds, 60)
uren, minuten = divmod(minuten, 60)
print "uren: {}\nminuten: {}\nseconden: {}".format(uren, minuten, seconden)

aantal seconden: 14185
uren: 3
minuten: 56
seconden: 25


In [13]:
# Dit kunnen we nu ook in de loop opnemen:
from datetime import datetime
for line in filelines:
    # Eerst maken we van de regel een lijst
    line_list = line.split()
    
    # Item 0 t/m 3 is de datum.
    # Ter herinnering, de regel ziet er als volgt uit:
    # Sep 16 10:38:45 yoram-thinkpad kernel: [ 1948.146436] CPU0: Package temperature/speed normal
    date_string = ' '.join(line_list[:3])
    date_object = datetime.strptime(date_string, '%b %d %H:%M:%S')
    time_delta = datetime.now() - date_object
    
    hostname = line_list[3]
    
    # Vanaf het 5e item begint de message
    message = ' '.join(line_list[4:])
    
    print "date: {} ({} seconden geleden)\nhostname: {}\nmessage: {}\n---------------".format(
        date_string, time_delta.seconds, hostname, message,
    )

date: Sep 16 10:38:45 (14203 seconden geleden)
hostname: yoram-thinkpad
message: kernel: [ 1948.146436] CPU0: Package temperature/speed normal
---------------
date: Sep 16 10:38:49 (14199 seconden geleden)
hostname: yoram-thinkpad
message: kernel: [ 1952.421197] mce: [Hardware Error]: Machine check events logged
---------------
date: Sep 16 10:39:57 (14131 seconden geleden)
hostname: yoram-thinkpad
message: wpa_supplicant[3370]: wlan0: CTRL-EVENT-SCAN-STARTED
---------------


## Configuratie files met ConfigParser

De file test.ini bevat de volgende info:

```
[sectie1]
key1 = waarde1
key2 = waarde2
key3 = waarde3

[sectie2]
key4 = waarde4
key5 = waarde5
key6 = waarde6
```

In [6]:
from ConfigParser import ConfigParser

config = ConfigParser()
config.read('test.ini')

['test.ini']

In [7]:
secties = config.sections()
print "Secties: {}".format(secties)
print "---------"

for sectie in config.sections():
    print "Sectie: {}".format(sectie)

    for item in config.options(sectie):
        print "{}: {}".format(item, config.get(sectie, item))
    
    print "--------"

Secties: ['sectie1', 'sectie2']
---------
Sectie: sectie1
key1: waarde1
key2: waarde2
key3: waarde3
--------
Sectie: sectie2
key4: waarde4
key5: waarde5
key6: waarde6
--------


## Data opslaan in json

### Vooraf

We hebben zonet het volgende commando gerunned:
```
find /tmp -exec md5sum {} \;
```

en hier is het volgende uitgekomen en weggeschreven naar sums.txt

```
48a6e19675ef52a879470cce58aac6a8  ./images/python_logo.png
d42c6e0efdf929036634176bbeae27b6  ./images/cern.jpeg
9c6bc947ba5eddef60e544fe9752590e  ./images/nasa.jpeg
1bbdcde81304d2d5575ae72848e31a38  ./images/yahoo.jpeg
bfe3688ea526e2700b8b38e1cdf59435  ./presentatie.ipynb
c3b3eb40d3f03d22fa27ec26fc5c0a8e  ./test.log
5f6f0b075568f2df58e2f1b9c8238630  ./Voorbeelden.ipynb
d05a415a94ad558f0489a5ef8bdf29a6  ./test.ini
```

## Inlezen in dictionary

In [8]:
import pprint

checksum_dict = {}

with open('sums.txt', 'r') as filehandle:
    for line in filehandle.readlines():
        checksum, filename = line.split()
        checksum_dict[checksum] = filename
        
pprint.pprint(checksum_dict)

{'1bbdcde81304d2d5575ae72848e31a38': './images/yahoo.jpeg',
 '48a6e19675ef52a879470cce58aac6a8': './images/python_logo.png',
 '5f6f0b075568f2df58e2f1b9c8238630': './Voorbeelden.ipynb',
 '9c6bc947ba5eddef60e544fe9752590e': './images/nasa.jpeg',
 'bfe3688ea526e2700b8b38e1cdf59435': './presentatie.ipynb',
 'c3b3eb40d3f03d22fa27ec26fc5c0a8e': './test.log',
 'd05a415a94ad558f0489a5ef8bdf29a6': './test.ini',
 'd42c6e0efdf929036634176bbeae27b6': './images/cern.jpeg'}


### JSON

In [9]:
import json

json_string = json.dumps(checksum_dict)

print json_string

{"c3b3eb40d3f03d22fa27ec26fc5c0a8e": "./test.log", "1bbdcde81304d2d5575ae72848e31a38": "./images/yahoo.jpeg", "d42c6e0efdf929036634176bbeae27b6": "./images/cern.jpeg", "d05a415a94ad558f0489a5ef8bdf29a6": "./test.ini", "9c6bc947ba5eddef60e544fe9752590e": "./images/nasa.jpeg", "5f6f0b075568f2df58e2f1b9c8238630": "./Voorbeelden.ipynb", "48a6e19675ef52a879470cce58aac6a8": "./images/python_logo.png", "bfe3688ea526e2700b8b38e1cdf59435": "./presentatie.ipynb"}


In [10]:
nieuwe_dict = json.loads(json_string)
pprint.pprint(nieuwe_dict)

{u'1bbdcde81304d2d5575ae72848e31a38': u'./images/yahoo.jpeg',
 u'48a6e19675ef52a879470cce58aac6a8': u'./images/python_logo.png',
 u'5f6f0b075568f2df58e2f1b9c8238630': u'./Voorbeelden.ipynb',
 u'9c6bc947ba5eddef60e544fe9752590e': u'./images/nasa.jpeg',
 u'bfe3688ea526e2700b8b38e1cdf59435': u'./presentatie.ipynb',
 u'c3b3eb40d3f03d22fa27ec26fc5c0a8e': u'./test.log',
 u'd05a415a94ad558f0489a5ef8bdf29a6': u'./test.ini',
 u'd42c6e0efdf929036634176bbeae27b6': u'./images/cern.jpeg'}


## Unieke bezoekers uit logfile

server.log bevat de volgende regels:

```
92.63.81.114 - - [27/Mar/2015:14:08:52 +0100] "GET / HTTP/1.1" 200 16833 "http://phaxa.nl/" "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1)" "-"
66.249.67.123 - - [27/Mar/2015:16:10:59 +0100] "GET /feed/ HTTP/1.1" 200 1563 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-"
--- etc ---
```

In [11]:
bezoekers = []

with open('server.log') as filehandle:
    for line in filehandle.readlines():
        bezoeker = line.split()[0]
        bezoekers.append(bezoeker)
        
for bezoeker in bezoekers:
    while bezoekers.count(bezoeker) > 1:
        bezoekers.remove(bezoeker)
        
# Dit is ook als volgt te doen:
# bezoekers = list(set(bezoekers))
        
print bezoekers

['94.76.241.6', '216.218.206.66', '92.63.81.114', '217.69.143.63', '185.27.174.170', '66.249.67.115', '66.249.67.131', '66.249.67.123']


## Hoe vaak komt een bezoeker voor?

In [12]:
bezoekers = {}

with open('server.log', 'r') as filehandle:
    for line in filehandle.readlines():
        bezoeker = line.split()[0]
        try:
            bezoekers[bezoeker] += 1
        except KeyError:
            bezoekers[bezoeker] = 1
            
for bezoeker in bezoekers.keys():
    print "{} heeft {} entries.".format(bezoeker, bezoekers[bezoeker])

185.27.174.170 heeft 3 entries.
94.76.241.6 heeft 1 entries.
66.249.67.123 heeft 3 entries.
66.249.67.131 heeft 1 entries.
217.69.143.63 heeft 1 entries.
216.218.206.66 heeft 1 entries.
92.63.81.114 heeft 2 entries.
66.249.67.115 heeft 3 entries.
